In [1]:
%matplotlib inline
#%pdb
#import time
#tstart = time.time()

In [2]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
from matplotlib import pyplot as plt
import wfirst_snhostspec as wfirst
from glob import glob

Read in the simulated SN data from the SNANA sim data files.
Each SNANA simulation has generated a HEAD.FITS file that contains a binary table with metadata for each simulated SN and host galaxy.  The high-z host galaxy magnitudes have been drawn from distributions that match the CANDELS+CLASH sample -- so there is some selection bias built in, but it will be similar to the selection biases of the WFIRST SN survey (?). 

In [3]:
simlist = []
simfilelist_med = glob('SNANA.SIM.OUTPUT/*Z08*HEAD.FITS')
simfilelist_deep = glob('SNANA.SIM.OUTPUT/*Z17*HEAD.FITS')
hostz_med, hostmag_med = np.array([]), np.array([])
for simfile in simfilelist_med:
    sim = wfirst.WfirstSimData(simfile)
    sim.load_matchdata('3DHST/3dhst_master.phot.v4.1/3dhst_master.phot.v4.1.cat.FITS')
    sim.get_matchlists()
    hostz_med = np.append(hostz_med, sim.zsim)
    hostmag_med = np.append(hostmag_med, sim.mag)
    simlist.append(sim)

hostz_deep, hostmag_deep = np.array([]), np.array([])
for simfile in simfilelist_deep:
    sim = wfirst.WfirstSimData(simfile)
    sim.load_matchdata('3DHST/3dhst_master.phot.v4.1/3dhst_master.phot.v4.1.cat.FITS')
    sim.get_matchlists()
    hostz_deep = np.append(hostz_deep, sim.zsim)
    hostmag_deep = np.append(hostmag_deep, sim.mag)
    simlist.append(sim)    

Now for each SNANA sim file, load in the catalog of galaxy SED data from 3DHST and use EAZY to simulate an SED.  The output simulated SEDs are stored in the sub-directory '3dHST/sedsim.output'

In [4]:
if not os.path.isdir('3DHST/sedsim.output'):
    os.mkdir('3DHST/sedsim.output')
for sim in simlist:
    sim.load_sed_data()
    sim.simulate_seds()

wfirst_snhostspec.py:207: RuntimeWarning: divide by zero encountered in double_scalars
  flam_spec = 1. / (1 + z) ** 2
wfirst_snhostspec.py:208: RuntimeWarning: invalid value encountered in multiply
  tempflux = templf * fnu_factor * flam_spec
wfirst_snhostspec.py:227: RuntimeWarning: divide by zero encountered in log10
  3.34e4 * wave * wave * f_lambda / 3631)


TODO NEXT : run the Subaru ETC on each simulated galaxy spectrum and determine the S/N achieved after 1 hour 5 hour, 10 hour exposures

In [ ]:
# Example of a spectrum plot
eazyspecsim = wfirst.EazySpecSim('3DHST/sedsim.output/wfirst_simsed.AEGIS.0185.dat')
eazyspecsim.plot()

In [ ]:
photdat3d = fits.open('3DHST/3dhst_master.phot.v4.1/3dhst_master.phot.v4.1.cat.FITS')
f160 = photdat3d[1].data['f_F160W']
zspec = photdat3d[1].data['z_spec']
zphot = photdat3d[1].data['z_peak']
zbest = np.where(zspec>0, zspec, zphot)
usephot = photdat3d[1].data['use_phot']

ivalid = np.where(((f160>0) & (zbest>0)) & (usephot==1) )[0]
mH3D = -2.5*np.log10(f160[ivalid])+25
z3D = zbest[ivalid]
plt.plot(z3D, mH3D, 'b.', ls=' ', ms=1, alpha=0.1)
#plt.plot(hostz_med, hostmag_med, 'g.', ls=' ', ms=3, alpha=0.3)
plt.plot(hostz_deep, hostmag_deep, 'r.', ls=' ', ms=3, alpha=0.3)
ax = plt.gca()
xlim = ax.set_xlim(0,2.5)
ylim = ax.set_ylim(28,20)
ax.set_xlabel('redshift')
ax.set_ylabel('host galaxy AB magnitude')

In [ ]:
ztestlist = [0.8,1.2,1.5,2.0,2.5]
for ztest in ztestlist:
    inear = np.where(np.abs(hostz_deep-ztest)<0.05)[0]
    magnear = hostmag_deep[inear]
    print("{:3.1f} {:4.1f} {:4.1f} {:4.1f}".format(
            ztest, magnear.min(), np.median(magnear), magnear.max()))

In [ ]:
sim1701.load_matchdata('3DHST/3dhst_master.phot.v4.1/3dhst_master.phot.v4.1.cat.FITS')
sim1701.get_matchlists(dz=0.02, dH=0.2)

In [ ]:
sim1701.matchdata.columns

In [ ]:
fieldid = sim1701.matchid[0]
fieldstr, idxstr = fieldid.split('.')
field = fieldstr.lower().replace('-','')
idx = int(idxstr)
print(field)
print(idx)
print(sim1701.matchid[0])

Generate a simulated host galaxy spectrum with EAZY 

In [ ]:
templz, tempmag = wfirst.simulate_eazy_sed(sim1701.matchid[0], savetofile='wfirst_simsed.dat')

TODO: run the ETC on the simulated spectrum

from the command line we would do: 

  python ~/src/subarupfsETC/run_etc.py @wfirst_subarupfs_etc.defaults

Read in the ETC output

In [ ]:
reload(wfirst)
etcdat = wfirst.SubaruSpecSim('etcout/ref.snc.dat')

In [ ]:
plt.plot(etcdat.wave, etcdat.signaltonoise,'k-', marker=' ')

In [ ]:
tfinish = time.time()
print('{:.3e} sec elapsed'.format(tfinish-tstart))

In [ ]:
fitsfilename = glob('3DHST/goodss_3dhst.*.eazypy.data.fits')[0]
hdu = fits.open(fitsfilename)
hdu.readall()
hdu.close()
hdu['ID'].data

In [ ]:
print len(templz)
print len(tempmag)
plt.plot(templz, tempmag, color='k', marker=' ', ls='-', lw=1)
ax = plt.gca()
ax.set_xlim(300,2100)
ax.set_ylim(30, 20)
print(templz.min())
print(templz.max())
print(tempmag.max())
print(tempmag.min())

In [ ]:
print('{:.1f} {:.1f} {:.1f} {:.1f} '.format(np.min(sim1701.nmatch),
        np.mean(sim1701.nmatch), np.median(sim1701.nmatch),
        np.std(sim1701.nmatch)))
plt.hist(sim1701.nmatch, bins=50)

In [ ]:
np.random.shuffle?

Read in the simulated SN data from the SNANA sim data files.
Each SNANA simulation has generated a HEAD.FITS file that contains a binary table with metadata for each simulated SN and host galaxy.  The high-z host galaxy magnitudes have been drawn from distributions that match the CANDELS+CLASH sample -- so there is some selection bias built in, but it will be similar to the selection biases of the WFIRST SN survey (?). 

The data we want to extract from the head.fits files are:  
#COL  LABEL
1 SNID
9 SNTYPE
22 HOSTGAL_PHOTOZ
23 HOSTGAL_PHOTOZ_ERR
24 HOSTGAL_SPECZ
25 HOSTGAL_SPECZ_ERR
27 HOSTGAL_LOGMASS
28 HOSTGAL_LOGMASS_ERR
XX HOSTGAL_MAG_R
XX HOSTGAL_MAG_Y 
XX HOSTGAL_MAG_J
XX HOSTGAL_MAG_H
XX HOSTGAL_MAG_F
49 SIM_REDSHIFT_HOST

In [ ]:
snanasimhead = fits.open(
    'SNANA.SIM.OUTPUT/IMG_2T_4FILT_MD_SLT3_Z17_Ia-01_HEAD.FITS')
zsim = snanasimhead[1].data['SIM_REDSHIFT_HOST']
hostmagH = snanasimhead[1].data['HOSTGAL_MAG_H']
fig = plt.figure(1)
ax1 = fig.add_subplot(111)
ax1.plot(zsim, hostmagH, marker='.', ls=' ', color='r')

From the 3DHST catalog, we extract redshift and H band mag for the actual galaxies in the 3DHST fields.  Below is an overlay plot showing both the SN sample and the 3DHST sample.

In [ ]:
photdat3d = fits.open('3DHST/3dhst_master.phot.v4.1/3dhst_master.phot.v4.1.cat.FITS')
f160 = photdat3d[1].data['f_F160W']
zspec = photdat3d[1].data['z_spec']
zphot = photdat3d[1].data['z_peak']
zbest = np.where(zspec>0, zspec, zphot)
usephot = photdat3d[1].data['use_phot']

ivalid = np.where(((f160>0) & (zbest>0)) & (usephot==1) )[0]
mH3D = -2.5*np.log10(f160[ivalid])+25
z3D = zbest[ivalid]
plt.plot(z3D, mH3D, 'b.', ls=' ', ms=1, alpha=0.1)
plt.plot(zsim, hostmagH, 'r.', ls=' ', ms=3, alpha=0.5)

#photdat3d[1].header